We provide all products and services under the terms of the Creative Commons Attribution-ShareAlike 4.0 International licence (CC BY-SA 4.0).

The data and database are open and licensed by the Open Data Commons Open Database License (ODbL).

Under these licences, you can freely use our products and data for non-commercial or commercial purposes and credit OpenWeather as a weather data source in a visible part of your application.

For an Enterprise account, we can work under customised licences. Please contact us to discuss your requirements and conditions.

In [2]:
import requests
import datetime as dt
from geopy.geocoders import Nominatim
import os
import numpy as np
import pandas as pd
import time

In [9]:
#%%writefile openweather_5days.py 
def OpenWeatherMap_5days(location_name=None, longitude=None, latitude=None, units='metric'):
    #import packages
    import requests
    import datetime as dt
    import time
    import geopy
    from geopy.geocoders import Nominatim
    import numpy as np
    import pandas as pd

    #determine latitiude and longitude based on what was passed to function (may change from what was )
    if location_name:
        locator = Nominatim(user_agent='myGeocoder')
        location = locator.geocode(location_name)
        lati = str(location.latitude)
        long = str(location.longitude)
    else: 
        lati = str(latitude)
        long = str(longitude)
    
    exclude='current,minutely,hourly'
    
    #define API key
    API_key = '617dec18b0669db53a5ab299c4e48609'
    
    output = pd.DataFrame()
    
    for i in range(0,6):
        #datetime = dt.datetime.strptime(date, '%m/%d/%Y')
        date = dt.date.today()-dt.timedelta(days=i)
        datetimeunix = int(time.mktime(date.timetuple()))
    
        #One Call: Current weather, Minute forecast for 1 hour, Hourly forecast for 48 hours, Daily forecast for 7 days
        #National weather alerts, Historical weather data for the previous 5 days
        url = r'''https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={time}&appid={API_key}'''.format(lat=lati, lon=long, API_key=API_key, time=datetimeunix)
    
        response = requests.get(url)
        scraped = response.json()
        
        hourly=pd.DataFrame(scraped['hourly'])

        hourly_1 = pd.concat([hourly.drop(['weather'], axis=1), hourly['weather'].apply(lambda x: pd.Series(x[0]))], axis=1)
        hourly_drop = ['feels_like','pressure', 'visibility','humidity','dew_point','clouds','wind_speed', 'wind_gust', 'wind_deg','id','icon']
        for col in hourly_drop:
            if col in hourly_1.columns:
                hourly_1.drop(columns=[col], inplace=True)
        hourly_1['dt'] = pd.to_datetime(hourly_1['dt'], unit='s')
        if 'rain' not in hourly_1.columns:
            hourly_1['rain']=0
        if 'snow' not in hourly_1.columns:
            hourly_1['snow']=0
        hourly_1.insert(1, value=scraped['lat'], column='latitude')
        hourly_1.insert(1, value=scraped['lon'], column='longitude')
        hourly_1.rename(columns={'dt':'date', 'main':'weather_short','description':'weather_long'}, inplace=True)
                
        if i==0:
            output = hourly_1
        else:
            output = pd.concat([output,hourly_1])
        
    #add prefix for specific scraper
    output = output.add_prefix('owm_')
    #correct date column to date for mathcing with others
    output.rename(columns={'owm_date':'date'}, inplace=True)
        
    return output
    
    

Writing openweather_5days.py


In [8]:
scraped = OpenWeatherMap_5days(location_name='cairo',latitude=34, longitude =-70)

scraped

,date,owm_longitude,owm_latitude,owm_temp,owm_weather_short,owm_weather_long,owm_rain,owm_snow
0,2021-05-26 00:00:00,31.2357,30.0444,296.03,Clear,clear sky,0,0
1,2021-05-26 01:00:00,31.2357,30.0444,295.31,Clear,clear sky,0,0
2,2021-05-26 02:00:00,31.2357,30.0444,293.97,Clear,clear sky,0,0
3,2021-05-26 03:00:00,31.2357,30.0444,293.55,Clear,clear sky,0,0
4,2021-05-26 04:00:00,31.2357,30.0444,294.05,Clear,clear sky,0,0
...,...,...,...,...,...,...,...,...
19,2021-05-21 19:00:00,31.2357,30.0444,298.15,Clear,clear sky,0,0
20,2021-05-21 20:00:00,31.2357,30.0444,297.10,Clear,clear sky,0,0
21,2021-05-21 21:00:00,31.2357,30.0444,296.04,Clear,clear sky,0,0
22,2021-05-21 22:00:00,31.2357,30.0444,295.54,Clear,clear sky,0,0
